In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11792,2024-04-14,Eua Nba,16:30,Los Angeles Clippers,Houston Rockets,2.41,1.61,219.5,1.86,1.95,1.5,2.00,1.63,dCqVL441,0.414938,0.621118,0.537634,0.512821,0.036056,0.6,0.6,NaN,NaN,216.646,35.388537,0.163347,203.084,38.889971,0.191497,196.598,170.584,260.28,175.16,0.0,0.0,0.0,0.0,0.281436,0.033407,0.144148,-3.48,-0.696,-2.025862,0.681539,0.5,-0.181539,1.13,0.226,2.699115,0.521106,0.4,-0.121106
11793,2024-04-14,Eua Nba,16:30,Minnesota Timberwolves,Phoenix Suns,1.79,2.09,216.5,1.90,1.96,-2.5,2.00,2.32,nwAoUSdL,0.558659,0.478469,0.526316,0.510204,0.037128,0.4,0.8,NaN,NaN,205.032,28.381116,0.138423,182.692,34.277753,0.187626,186.938,217.270,197.22,168.48,0.0,0.0,0.0,0.0,0.109346,0.021983,0.104757,-1.95,-0.390,-2.025641,0.655379,0.4,-0.255379,0.06,0.012,90.833333,0.570698,0.5,-0.070698
11794,2024-04-14,Itália Liga A,15:00,Scafati,Varese,1.61,2.33,173.5,1.81,1.93,-4.5,2.04,3.05,WbJ7x2qM,0.621118,0.429185,0.552486,0.518135,0.050303,0.8,0.4,NaN,NaN,219.512,166.279280,0.757495,221.522,57.452676,0.259354,145.792,231.390,114.76,232.68,0.0,0.0,0.0,0.0,0.258435,0.045376,0.280620,-0.01,-0.002,-305.000000,0.522821,0.7,0.177179,2.85,0.570,2.333333,0.000000,0.0,0.000000
11795,2024-04-14,Alemanha Bbl,10:30,Bamberg,Hamburg,1.61,2.33,176.5,1.81,1.95,-3.5,1.92,3.05,8Gtra55G,0.621118,0.429185,0.552486,0.512821,0.050303,0.4,0.2,NaN,NaN,176.120,39.589403,0.224787,180.886,73.651914,0.407173,150.788,213.254,119.56,171.72,0.0,0.0,0.0,0.0,0.258435,0.052657,0.321542,-1.18,-0.236,-2.584746,0.000000,0.0,0.000000,0.72,0.144,9.236111,0.370798,0.6,0.229202
11796,2024-04-14,Alemanha Bbl,13:30,Wurzburg,Bayern,2.47,1.55,161.5,1.86,1.92,3.5,1.96,1.29,tUvjcqzT,0.404858,0.645161,0.537634,0.520833,0.050020,0.4,0.6,NaN,NaN,182.450,56.072395,0.307330,116.660,17.672838,0.151490,157.364,124.856,142.48,128.76,0.0,0.0,0.0,0.0,0.323651,0.022448,0.291546,3.89,0.778,1.889460,0.000000,0.0,0.000000,0.37,0.074,7.432432,0.000000,0.0,0.000000
11797,2024-04-14,Alemanha Pro A,11:00,Vechta 2,PS Karlsruhe,2.09,1.69,169.5,1.87,1.79,1.5,1.94,1.54,jit8Wkbo,0.478469,0.591716,0.534759,0.558659,0.070185,0.8,0.4,NaN,NaN,208.254,86.849446,0.417036,156.462,49.111044,0.313885,231.490,167.966,245.03,139.87,0.0,0.0,0.0,0.0,0.149652,0.030912,0.162553,2.87,0.574,1.898955,0.000000,0.0,0.000000,1.56,0.312,2.211538,0.000000,0.0,0.000000
11798,2024-04-14,Argentina Liga A,20:30,Penarol,San Martin,1.86,1.86,149.5,1.83,1.85,-1.5,2.01,2.01,CKLIo3Oh,0.537634,0.537634,0.546448,0.540541,0.075269,0.8,0.6,NaN,NaN,184.514,64.988844,0.352216,143.776,44.660780,0.310628,174.490,157.564,133.90,153.26,0.0,0.0,0.0,0.0,0.000000,0.007686,0.000000,1.95,0.390,2.205128,0.000000,0.0,0.000000,0.56,0.112,7.678571,0.508190,0.5,-0.008190
11799,2024-04-14,Austrália Nbl1 Norte,00:00,Gold Coast Rollers F,South West Metro Pirates F,1.79,1.93,147.5,1.85,1.85,-1.5,1.90,1.89,Oba5VEYp,0.558659,0.518135,0.540541,0.540541,0.076794,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.053223,0.000000,0.003731,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11800,2024-04-14,Austrália Nbl1 Sul,02:00,Casey Cavaliers,Bendigo Braves,1.95,1.77,169.5,1.83,1.87,-3.5,2.04,2.04,Ojg4YLNf,0.512821,0.564972,0.546448,0.534759,0.077792,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,150.15,0.00,0.0,0.0,0.0,0.0,0.068430,0.015289,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11801,2024-04-14,Aus

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,16:30,Eua Nba,Minnesota Timberwolves,Phoenix Suns,216.5,1.90,1.0000,Over
1,15:00,Itália Liga A,Scafati,Varese,173.5,1.81,1.0000,Over
2,10:30,Alemanha Bbl,Bamberg,Hamburg,176.5,1.81,1.0000,Over
3,20:30,Argentina Liga A,Penarol,San Martin,149.5,1.83,1.0000,Over
4,02:00,Austrália Nbl1 Sul,Casey Cavaliers,Bendigo Braves,169.5,1.83,1.0000,Over
5,03:00,Austrália Nbl1 Sul,Keilor Thunder,Geelong,180.5,1.85,0.9780,Over
6,08:35,China Cba,Guangzhou,Qingdao,197.5,1.87,1.0000,Over
7,15:00,Espanha Leb Ouro,Castello,Hestia Menorca,156.5,1.85,1.0000,Over
8,10:00,Europa Liga Bnxt,Liege,Groningen,156.5,1.87,1.0000,Over
9,10:00,Europa Euroliga Feminina,CBK Mersin F,USK Prague F,148.5,1.86,1.0000,Over
